In [1]:
#!pip install modestga --upgrade --quiet
#!pip install deap --upgrade --quiet
import warnings
warnings.filterwarnings("ignore")

In [2]:

import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import pickle
import time
# Start timer
start_time = time.perf_counter()


In [3]:
#===========================================================
#                   Read Excel File (Node_Property)
#===========================================================
#!!!!Notice: Name of the nodes existing in the Node Name column must be in number format not string
# such as 201, 202, 203 Not cmp[201], ...

file = pd.ExcelFile('Nodes-Property.xlsx')
df1=pd.read_excel(file,'Node_Property')
data=df1[df1.columns[:]].to_numpy()
data = np.unique(data,axis=0)
#print ('data: \n',data)
Node_Name= data[:,0]
#print('Node_Name=', Node_Name)
Memory= data[:,1]                                # Accessible memory size of CNk for running tasks
tsk= data[:,2]                                   # Switch time between two jobs for CNk 
#print(tsk)
#===========================================================
#                   Read Excel File
#===========================================================

file = pd.ExcelFile('results_bacteria88g_4nodes.xlsx')
#file = pd.ExcelFile('results_archea1.4g_24nodes.xlsx')
df1=pd.read_excel(file,'General-Results')
data=df1[df1.columns[:]].to_numpy()
data = np.unique(data,axis=0)
#print ('data: \n',data)
nodename= data[:,0]
x= data[:,1]
y= data[:,2]

#print('nodename=',nodename )
nodename_item =list(dict.fromkeys(nodename))
#print('nodename_item=' ,nodename_item )
V = len(nodename_item)  # The number of nodes of the cluster

#===========================================================
#                   Read Excel File 
#===========================================================
#genomes_df= pd.read_csv('overview-test.xlsx')
genomes_df= pd.read_excel('overview_results_bacteria88g.xlsx')
#genomes_df= pd.read_excel('overview_results_archea1.4g.xlsx')


#===========================================================
#                   Initializing Variables
#===========================================================

#CNk=0                             # kth node of the cluster 
#Memk=0                            # Accessible memory size of
                                   # Switch time between two jobs for CNk
n=len(genomes_df.GenomeName)       # The number of genomes of the reference genome
Gen= genomes_df.GenomeName         # 
Gen= Gen[0:n]                      # The set of genomes of the reference genome
Si=genomes_df.Size                 # 
Si= Si[0:n]                        # Size of gi
#J = n                           # The maximum number of Jobs
J = int(V*5)
print('#J=',J)
Dictionary={}
FinalOutput={}

#===========================================================
#                   Initializing Genetic Algorithm parameters 
#===========================================================

solution =[]

population_number=30
MutationRate= 0.4
CrossoverRate= 1 - MutationRate
#maxIter = 120
maxIter = 20
MutationNumber = round (population_number* MutationRate);
CrossoverNumber = round (population_number * CrossoverRate);


#J= 20


In [4]:
def fk (k,x):
    #loaded_model = pickle.load(open('heart-disease-model.pk1' , 'rb'))
    x = np.array([x]).reshape(-1, 1)
    linear_reg=pickle.load(open(FinalOutput[k]['filename'], 'rb')) 
    return linear_reg.predict(x)
    
#================================================================================================
#================================================================================================

for i in range (V):
  #print('i : ', i)
  a= data ==nodename_item[i]
  nodenamep= nodename[a[:,0]]
  xp= x[a[:,0]]
  yp= y[a[:,0]] 
    
#pickle.dump(model , open('heart-disease-model.pk1' , 'wb'))
  
  model = LinearRegression()
  xp = xp.reshape(-1, 1)
  model.fit(xp, yp)  
  intNode_Name= nodenamep[0].astype(int)
  strNode_Name= intNode_Name.astype(str)
  filename=str(strNode_Name)+".pkl"
  pickle.dump(model, open(filename, 'wb'))
    
  Dictionary={}
  Dictionary["NodeName"] = int(nodenamep[0])
  Dictionary["filename"] = filename
  #Dictionary["linear-spline"]=y_linear   
  FinalOutput[i]= Dictionary

#print('\n')
#print('\n')
#print('\n')

#FinalOutput






In [5]:
'''for i in range (V):
    print(FinalOutput[i]['filename'])'''
    
    

"for i in range (V):\n    print(FinalOutput[i]['filename'])"

In [6]:
'''for i in range (V):
    print(fk(i,10))'''



'for i in range (V):\n    print(fk(i,10))'

In [7]:
def jobs2cluster(S):
    AssignedJobs2Cluster={}
    for i in range (V):
        AssignedJobs2Cluster[i]=[]
    for j in range (n):
        Jobj=int(S[0][j])
        CNk=int(S[1][j])
        AssignedJobs2Cluster[CNk].append(Jobj)
    return (AssignedJobs2Cluster)

In [8]:
def genome2cluster(SS):
    g2c={}
    for i in range (V):
        g2c[i]=[]
    for j in range (n):
        Genomej=j
        CNk=int(SS[1][j])
        g2c[CNk].append(Genomej)
    return(g2c.copy())

In [9]:
"""Feasibility function for the Memk. Returns True if feasible False
    otherwise."""
def feasibleMemk(sol):
 
    
    M=np.zeros((V,J))

        
   
    for i in range (n):
        jobj= int(sol[0][i])
        CNk = int(sol[1][i])
        
        item= Si[i]
        q= len(item)
        if item[q-1]== 'K' or item[q-1]== 'k' :
            DigitSize=item[:-1]
            
            M[CNk][jobj]+= (float(DigitSize)/1000)
        elif  item[q-1]== 'M' or item[q-1]== 'm' :
            DigitSize=item[:-1]
            M[CNk][jobj]+= (float(DigitSize))
        
    for k in range (V):
        for j in range (J):
            if  Memory[k]< 3*M[k][j]:
                return False
    return True
    
  

In [10]:
def ComputeJobSize(g2j):
    sm=[]
    for j in range (J):
            sm.append(0.0)
    for j in range (J):

            GenomesNumber= g2j[j]
            m= len(GenomesNumber)
            #print(" GenomesNumber:",  GenomesNumber)

            #extract memory size of each genome belonged to Job j
            MemoryItems = (Si[GenomesNumber])
            #print (' MemoryItems :',  MemoryItems )
            for i in GenomesNumber:
                item= Si[i]
                #print ('Genome Size {} is: {}'.format(i,Si[i]))
                q= len(item)
                if item[q-2]== 'K' or item[q-2]== 'k' :
                    DigitSize=item[:-1]
                    sm[j]+= float(DigitSize)/1000
                elif  item[q-2]== 'M' or item[q-2]== 'm' :
                    DigitSize=item[:-2]
                    sm[j]+= float(DigitSize)
    print('Compute job size=',sm.copy() )
    return sm.copy()

In [11]:
'''def RunningTime (Sl,g2cn):
    tk= np.zeros((V,1))
    J2C=jobs2cluster(Sl)
    G2J=Genomes2Jobs(Sl)
    JS= ComputeJobSize(G2J) # Job Size
    for k in range (V):
        NeedMemory=0
        for j in range (len(np.unique(J2C[k]))):
            #print ('Cluster Number is: {},  Job Number is {}, and Job size is {}. \n '.format (k,J2C[k][j],JS[J2C[k][j]]))
            NeedMemory=JS[J2C[k][j]]
            #tk[k]+=fk(FinalOutput[k]['BestPloynomialDegree'],NeedMemory,FinalOutput[k]['Coefficients'])
            if(NeedMemory == 0) :
                #print("error 0") 
                continue
            else:
                tk[k]+=fk(k,NeedMemory)

    for i in range (V):
        GenomesNumber= g2cn[i]
        m= len(np.unique(J2C[i]))
        tk[i]+= ((m-1)*tsk[i])
        
    print('tk=',max(tk))    
    
    return max(tk)
'''
def RunningTime (Sl,g2cn):
    tk= np.zeros((V,1))
    J2C=jobs2cluster(Sl)
    G2J=Genomes2Jobs(Sl)
    JS= ComputeJobSize(G2J) # Job Size
    for k in range (V):
        NeedMemory=0
        for j in range (len(np.unique(J2C[k]))):
            #print ('Cluster Number is: {},  Job Number is {}, and Job size is {}. \n '.format (k,J2C[k][j],JS[J2C[k][j]]))
            NeedMemory=JS[J2C[k][j]]
            if(NeedMemory == 0) :
                print("error 0")
                continue
            else:
                tk[k]+=fk(k,NeedMemory)
        print('tk[',k,']=',tk[k])
    '''for i in range (V):
        GenomesNumber= g2cn[i]
        m= len(np.unique(J2C[i]))
        tk[i]+= ((m)*tsk[i])
    #if (max(tk)== -10):
        print(' #jobs assigned to Node[',i,']=',m)'''
    print('tk=',max(tk))
    return max(tk)


 
          
        
    
       

In [12]:
def Genomes2Jobs(SL):
    Ge2Jo={}
    for i in range (n):
        Ge2Jo[i]=[]
    for j in range (n):
        Genomej=j
        Jobj=int(SL[0][j])
        Ge2Jo[Jobj].append(Genomej)
    return(Ge2Jo.copy())

In [13]:
def clusters2job(solu):
    C2Job={}
    for i in range (n):
        C2Job[i]=[]
    for j in range (n):
        Jobj=int(solu[0][j])
        CNk=int(solu[1][j])
        C2Job[Jobj].append(CNk)
      
    return(C2Job.copy())


In [14]:
import random
def AddressJobs2ClustersProblem(SOL):
    Temp= clusters2job(SOL.copy())
    t= len(Temp)
    for j in range (len(Temp)):
        if len(Temp[j])>1:
            newCNkIndex= random.randint(0,len(Temp[j])-1)
            newCNk= Temp[j][newCNkIndex]
            for i in range (n):
                if SOL[0][i]== j:
                    SOL[1][i]=newCNk
    return SOL.copy()
                
           
        
    
    
    


In [15]:
def equalSols(S1,S2):
    for j in range(len(S1)):
        if (S1[1][j]!= S2[1][j] or S1[0][j]!= S2[0][j] ):
                return False
    return True

In [16]:
def isRepeated(POp,Sol):
    index= False
    newPop=[]
    for i in range(len(POp)):
        newPop.append((POp[i].copy()))
    newPop=np.array(newPop.copy())
    N= len(POp)
    for i in range(N):
        if (equalSols(Sol.copy(),newPop[i,:,:].copy())):
                index =True
    return index

In [17]:
def arrangeSolutions(POPU):
    newFit = []
    newPop=np.empty((len(POPU), 2,n))
    for i in range(len(POPU)):
         if (feasibleMemk(POPU[i].copy())==True):
           
            newPop[i]=((POPU[i].copy()))
            SolvedSol= AddressJobs2ClustersProblem(POPU[i].copy())   
            G2CNK= genome2cluster(SolvedSol.copy())
            Fit= RunningTime(SolvedSol.copy(),G2CNK)
            newFit.append(Fit)
    
    return newPop.copy(),newFit.copy()
        
    

In [18]:
def AddNewSol(POPulation,Sol,FITS,Fit):
    index= isRepeated(POPulation.copy(),Sol.copy())
    L= len( POPulation.copy())
    if(index == False ):
        L=L+1
        POPulation= np.append(POPulation.copy(),Sol.copy())
        POPulation= POPulation.reshape(L,2,n)
        FITS.append(Fit)
        
    return POPulation.copy(),FITS.copy()

In [19]:
def SortPopulation(poP, FS,Npop):
    
    newFit = []    
    newPop=[]
    
    for i in range(len(poP)):
        newPop.append((poP[i].copy()))
        newFit.append(FS[i][0].copy())
    s = np.array(newFit.copy())
   
    sort_index = np.argsort(newFit.copy())
    for i in range(len(poP)):
        newPop[i]=list( poP[sort_index[i]].copy())
    FS.sort()
    
    return newPop.copy(),FS.copy(),len(newPop.copy())

In [20]:

def CreatePopulation (n):
    #population[i][j][k] ==> i th population 
    #[row0] ==> Job of solution [.......]
    #[row1] ==> CNk of solution [.......]
    Npop=1
    NS= np.zeros((2,n))
    for i in range (n):
            NS[0,i]=int(random.randint(0,J-1))# Random Job  number
            NS[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
    NS= AddressJobs2ClustersProblem(NS.copy())   
    Sol= genome2cluster(NS.copy())
        
    #JobSize = totalMemk(Sol)
    if (feasibleMemk(NS.copy())==True):
            POPULATION= np.empty((1, 2,n))
            for i in range(Npop):
                 POPULATION[0]= NS.copy()
            Fit= RunningTime(NS.copy(),Sol)
    else:
            while(feasibleMemk(NS.copy())==False):
                NS= np.zeros((2,n))
                for i in range (n):
                    NS[0,i]=int(random.randint(0,J-1))# Random Job  number
                    NS[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
                NS= AddressJobs2ClustersProblem(NS.copy())   

                Sol= genome2cluster(NS.copy())

                #JobSize = totalMemk(Sol)
                
                
                if (feasibleMemk(NS.copy())==True):
                    
                    POPULATION= np.empty((1, 2,n))
                    for j in range(n):
                        POPULATION[0]= NS.copy()
                    Fit= RunningTime(NS.copy(),Sol)
            
    return POPULATION.copy(),Fit.copy()
        
    

In [21]:
def Mutation(PP,FITS):   
    P=PP.copy()
    FF=FITS.copy()
    for iMut in range(MutationNumber):
            Npop= len(PP)
            SelectedSolutionIndex= random.randint(0,len(P)-1)
            SelectedSolution= (P[SelectedSolutionIndex]).copy()
            #print("SelectedSolution:\n", SelectedSolution)

            SelectedGenomeIndex1 = random.randint(0,n-1)
            SelectedGenomeIndex2 = random.randint(0,n-1)

            tempJ=SelectedSolution[0][SelectedGenomeIndex1]
            tempCNk=SelectedSolution[1][SelectedGenomeIndex1]
            SelectedSolution[0][SelectedGenomeIndex1]= SelectedSolution[0][SelectedGenomeIndex2].copy()
            SelectedSolution[1][SelectedGenomeIndex1]= SelectedSolution[1][SelectedGenomeIndex2].copy()
            SelectedSolution[0][SelectedGenomeIndex2]= tempJ
            SelectedSolution[1][SelectedGenomeIndex2]= tempCNk
            MutedSolution = AddressJobs2ClustersProblem(SelectedSolution.copy())
            Sol= genome2cluster(MutedSolution.copy())
            
           


            if (feasibleMemk(MutedSolution.copy())==True and  isRepeated(P.copy(),MutedSolution.copy()) ==False):
                #print('Mut len:',len(MutedSolution))
                F=( RunningTime(MutedSolution.copy(),Sol.copy()))
                P, FF= AddNewSol(P.copy(), MutedSolution.copy(),FITS.copy(),F) 
                if len(P)!= len(FF):
                    P,F= arrangeSolutions(P)
                    #print("Pop:\n", PP)
                    #print("!!!Mutelen(P):",len(P))
                    #print ("len Fit",len(F))
    return P, FF



In [22]:
def Crossover (POP,FITN):
    PoP=POP.copy()
    FiT=FITN.copy()
    for iCross in range(CrossoverNumber):
            Npop= len(PoP)
            SelectedSolutionIndex1= random.randint(0,len(PoP)-1)
            SelectedSolutionIndex2= random.randint(0,len(PoP)-1)

            SS1= PoP[SelectedSolutionIndex1].copy()
            SS2= PoP[SelectedSolutionIndex2].copy()

            SelectedGenomeIndex = random.randint(0,n-1)
            temp=  SS2[SelectedGenomeIndex:n-1].copy()
            SS2[SelectedGenomeIndex:n-1]= SS1[SelectedGenomeIndex:n-1].copy()
            SS1[SelectedGenomeIndex:n-1]= temp.copy()


            SS1=AddressJobs2ClustersProblem(SS1.copy())
            Sol1= genome2cluster(SS1.copy())



            #JobSize = totalMemk(Sol1)
            if (feasibleMemk(SS1)==True and isRepeated(PoP.copy(),SS1.copy()) ==False):
                #print('Cross1 len:',len(SS1))
                F=(RunningTime(SS1.copy(),Sol1.copy()))
                PoP,FiT= AddNewSol(PoP.copy(), SS1.copy(),FiT.copy(),F)  
                


            SS2=AddressJobs2ClustersProblem(SS2)
            Sol2= genome2cluster(SS2)
            #JobSize = totalMemk(Sol2)
            if (feasibleMemk(SS2.copy())==True and isRepeated(PoP.copy(),SS2.copy()) ==False):
                #print('Croos2 len:',len(SS2))
                F=(RunningTime(SS2.copy(),Sol2.copy()))
                PoP,FiT= AddNewSol(PoP.copy(), SS2.copy(),FiT.copy(),F) 
    if (len(PoP)!= len(FiT)):
        PoP,FiT= arrangeSolutions(PoP.copy())
        #print("Pop:\n", PP)
        #print("!!!Crosslen(P):",len(PoP))
        #print ("len Fit",len(FiT))
    return PoP.copy(),FiT.copy()

In [23]:
#==================================================================
#                 Genetic Algorithm 
#==================================================================
    
def GA_Scheduling():
    #==================================================================
    #                 Create Initial Population
    #===================================================================
    Population=[]
    MaxPopN= 1000
    Npop=0  # number of added acceptable solutions to population
    fits=[]

    Population,Fit= CreatePopulation (n)
    fits.append(Fit)
    #print('initial Population',Population)
    while (Npop <population_number):
        for j in range (population_number):
            NSOL= np.zeros((2,n))
            for i in range (n):
                NSOL[0,i]=int(random.randint(0,J-1))# Random Job  number
                NSOL[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
            NSOL= AddressJobs2ClustersProblem(NSOL)   

            Sol= genome2cluster(NSOL)
            #JobSize = totalMemk(Sol)
            if (feasibleMemk(NSOL)==True and isRepeated(Population.copy(),NSOL.copy()) ==False):
                Fit= (RunningTime(NSOL,Sol))
                Population, fits= AddNewSol(Population.copy(),NSOL.copy(),fits.copy(), Fit)
                Npop+=1
    #print('initpop:',Population)
   
    #==================================================================
    #                 Genetic Algorithm 
    #===================================================================
    for iteration in range(maxIter):
        changed=False

        changed=False
        nMut=0   # number of acceptable Muted solutions 
        nCross=0 # number of acceptable Crossover solutions 
        Npop= len(Population)

        #for i in range (len (Population)):
            #print(Population[i])
        #==================================================================
        #                 Mutation
        #===================================================================
                
        #print ('Mut1:')
        #print(len(Population))
        #print(len(fits))
        Population,fits= Mutation(Population.copy(),fits.copy())
        #print ('Mut2:')
        #print(len(Population))
        #print(len(fits))
        #for i in range (len (Population)):
            #print(Population[i])
        #==================================================================
        #                Crossover
        #===================================================================
        #print ('Cros1:')
        #print(len(Population))
        #print(len(fits))
        Population,fits=Crossover (Population.copy(),fits.copy())
        #print ('Cros2:')
        #print(len(Population))
        #print(len(fits))

        #==================================================================
        #                Add Muted Solutions to Population
        #===================================================================

        ''' print ('Pop1:')
        for i in range (len (Population)):
            print(Population[i])'''
        Population, fits,Npop=SortPopulation(Population.copy(), fits.copy(), Npop-1)    
        Population,fits= arrangeSolutions(Population.copy())
        
        '''print ('Pop2:')
        for i in range (len (Population)):
            print(Population[i])'''
        '''print("\n***************************************************************************\n")
        print('Best Solution is of iteration of {} is:\n{} '.format(iteration,Population[0].copy() ))
        print("-----------------------------------------------------------------------------\n")'''
    
    return Population.copy(), fits.copy()

P, F= GA_Scheduling()

Compute job size= [4788.300000000002, 4283.800000000003, 4587.499999999995, 4413.600000000003, 4514.699999999994, 4501.9999999999945, 4596.7000000000035, 4500.699999999999, 4468.200000000003, 4725.000000000004, 4611.500000000002, 4656.100000000008, 4506.6, 4651.799999999993, 4662.499999999998, 4401.899999999999, 4417.799999999998, 4611.199999999996, 4375.599999999994, 4503.999999999998]
tk[ 0 ]= [12868.13120381]
tk[ 1 ]= [19745.10360697]
tk[ 2 ]= [16020.56208588]
tk[ 3 ]= [18686.10638804]
tk= [19745.10360697]
Compute job size= [4372.0, 4904.500000000004, 4709.000000000002, 4651.000000000007, 4700.4999999999945, 4329.399999999993, 4470.899999999991, 4511.700000000003, 4552.100000000001, 4479.599999999997, 4519.4, 4625.9000000000015, 4721.0, 4476.199999999997, 4396.800000000003, 4654.300000000002, 4405.099999999999, 4495.799999999995, 4535.7999999999965, 4268.5]
tk[ 0 ]= [22545.39110066]
tk[ 1 ]= [13012.70174342]
tk[ 2 ]= [19784.09963088]
tk[ 3 ]= [12767.95872228]
tk= [22545.39110066]
Co

In [24]:
BestSolution = P[0]
ClusterName= np.unique(nodename)
print('The best solution:\n')
for i in range(n):
    j= int(BestSolution[0][i])
    k= int(BestSolution[1][i])
    #print( "Genome {} is allocated to Job Number {} on Cluster Node Number {}.".format(i,j,ClusterName[k]))

print('\n and its fitness function is:\n',(F[0]))

The best solution:


 and its fitness function is:
 [18601.95701912]


In [25]:
J2C= jobs2cluster(BestSolution)
EmptyClusters=[]
for k in range(V):
    if (J2C[k]!= []):
        print('Cluster {} includes a subset of Job numbers:\n{}'.format(ClusterName[k],np.unique( J2C[k])))
    else:
        EmptyClusters.append(ClusterName[k])
print("================================================================================")
print ("Empty Clusters are:", (EmptyClusters))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

#print('Size Job number{} = {}'.format(np.unique( J2C[k]), )


Cluster 217.0 includes a subset of Job numbers:
[ 6  8 10 13 16]
Cluster 221.0 includes a subset of Job numbers:
[ 5  7  9 12 17]
Cluster 241.0 includes a subset of Job numbers:
[ 0  3 14 15]
Cluster 242.0 includes a subset of Job numbers:
[ 1  2  4 11 18 19]
Empty Clusters are: []
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


In [26]:
G2J= Genomes2Jobs(BestSolution)
JobSize=ComputeJobSize(G2J)
EmptyJobs=[]
for i in range(J):
    if (G2J[i]!= []):
        print('Job {} includes a subset of genomes numbers:{}'.format(i,G2J[i]))
    else:
        EmptyJobs.append(i)
print("================================================================================")
print ("Empty Jobs are:", EmptyJobs)
print ("The number of Empty Jobs is:", len(EmptyJobs))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
for k in range(J):
    print('Size Job number{} = {}'.format(k,JobSize[k]))
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print('running time of the best solution=' )

Compute job size= [4573.400000000005, 4368.399999999996, 4483.99999999999, 4483.100000000004, 4732.399999999997, 4702.400000000002, 4409.700000000001, 4540.699999999999, 4598.600000000002, 4561.000000000004, 4658.199999999996, 4465.700000000005, 4794.599999999995, 4609.9999999999945, 4509.500000000005, 4486.9000000000015, 4450.299999999989, 4569.699999999999, 4361.299999999996, 4419.600000000001]
Job 0 includes a subset of genomes numbers:[63, 67, 135, 138, 146, 162, 173, 216, 252, 305, 308, 332, 344, 353, 387, 434, 436, 449, 476, 484, 492, 510, 514, 532, 534, 538, 550, 554, 576, 589, 604, 631, 636, 647, 702, 738, 817, 826, 918, 962, 980, 984, 988, 1011, 1033, 1048, 1117, 1135, 1147, 1165, 1194, 1251, 1265, 1269, 1275, 1276, 1290, 1306, 1324, 1348, 1356, 1370, 1373, 1384, 1396, 1468, 1471, 1479, 1491, 1492, 1527, 1544, 1555, 1567, 1585, 1612, 1656, 1663, 1727, 1735, 1738, 1743, 1748, 1812, 1816, 1851, 1858, 1904, 1922, 1945, 1948, 1949, 1957, 1958, 2031, 2050, 2062, 2065, 2091, 2116, 2

In [27]:
# End timer
end_time = time.perf_counter()
print('\n and its fitness function is:\n',int(F[0]))
print("================================================================================")
# Calculate elapsed time
elapsed_time = end_time - start_time
print("Elapsed time: ", int(elapsed_time))
print("================================================================================")
print ("Empty Clusters are:", len(EmptyClusters))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print ("The number of Empty Jobs is:", len(EmptyJobs))


 and its fitness function is:
 18601
Elapsed time:  264
Empty Clusters are: 0
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
The number of Empty Jobs is: 0
